In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from string import ascii_lowercase
import itertools
import datetime
from timeit import default_timer as timer

# Data preparation 
## Helper Functions
### IOP preprocessing functions

In [2]:
# Filter period type
    # PERIOD_TYPE_ID: Filter on type W for Week - data on weekly level
def filter_period_type(df, ptype='W'):
    print("Apply function: filter_period_type")
    mask=df.PERIOD_TYPE_ID==ptype
    print('INFO: Filtered out {} with different ptype than {}.'.format(sum(~mask), ptype))
    return df[mask]

# Filter for latest delivery sequence
def filter_old_seq(df):
    print("Apply function: filter_old_seq")
    max_seq=pd.DataFrame(df.groupby(['DELIVERY_PERIOD_ID', 'DELIVERY_TYPE_ID'])['DELIVERY_SEQ'].max())\
              .reset_index()   
    latest_df=max_seq.merge(df, on=['DELIVERY_PERIOD_ID', 'DELIVERY_TYPE_ID', 'DELIVERY_SEQ'], how='left')
    print('INFO: Filter for latest delivery sequence. Filtered out {} records.'.format(len(df)-len(latest_df)))
    return latest_df

# Delete columns with 0 Turnover and 0 Sales Volume
def filter_blind_items(df):
    print("Apply function: filter_blind_items")
    mask=(df.TURNOVER==0) & (df.SALES_VOLUME==0) #& (df.PRICE_ORG==0)
    print('INFO: Filtered out {} blind items.'.format(sum(mask)))
    return df[~mask]

# Merge data: Join item master
def enrich_objects(df, path_item_md):
    print("Apply function: enrich_objects")
    objects=pd.read_csv(path_item_md, sep=';', decimal=',')\
              .assign(OBJECT_ID=lambda x: x['DETAIL_ID'])\
              .drop(columns='DETAIL_ID') 
    print('INFO: Join item master data.')
    return df.merge(objects, on='OBJECT_ID', how='left')

# Merge data: Join shop master
def enrich_shops(df, path_shop_md):
    print("Apply function: enrich_shops")
    shops=pd.read_csv(path_shop_md, sep=';', decimal=',')   
    print('INFO: Join shop master data.')
    return df.merge(shops, on=['DELIVERY_TYPE_ID', 'SHOP_ID'], how='left')

# Merge data: Join period master data and generate time stamps
    # PERIOD_TS: last date of KW
    # PERIOD_STRING: KW
def enrich_periods(df):
    print("Apply function: enrich_periods")
    periods=pd.read_csv('./data/master_data/IOP_DeliveryPeriods.csv', sep=';', decimal=',')\
              .set_index('PERIOD_ID')
    print('INFO: Join period master data and generate time stamps.')
    return df.assign(PERIOD_TS=pd.to_datetime(df['DELIVERY_PERIOD_ID'].map(periods.END_DATE), dayfirst=True))\
             .assign(PERIOD_STRING=(df['DELIVERY_PERIOD_ID'].map(periods.PERIOD_STRING)).astype(str))

# generate fill type if some values are 0
    # FILL_TYPE = 1 if nothing is 0
    # FILL_TYPE = 2 if Sales Volume is 0
    # FILL_TYPE = 3 if Turnover is 0
    # FILL_TYPE = 4 if Price_org is 0
    # FILL_TYPE = 0 if Only one Variable is not 0
def columns_fill_type(df):
    print("Apply function: columns_fill_type")
    A=(df.SALES_VOLUME!=0) & (df.TURNOVER!=0) & (df.PRICE_ORG!=0)
    B=(df.SALES_VOLUME==0) & (df.TURNOVER!=0) & (df.PRICE_ORG!=0)
    C=(df.SALES_VOLUME!=0) & (df.TURNOVER==0) & (df.PRICE_ORG!=0)
    D=(df.SALES_VOLUME!=0) & (df.TURNOVER!=0) & (df.PRICE_ORG==0)
    print('INFO: Fill type - Num(SV & TO & PR)={}, Num(TO & PR)={}, Num(SV & PR)={}, Num(SV & TO)={}, Num(Invalid)={}.'\
                  .format(sum(A), sum(B), sum(C), sum(D), sum(~(A | B | C | D))))
    return df.assign(FILL_TYPE=A.astype(int)+2*B.astype(int)+3*C.astype(int)+4*D.astype(int))


# How many days is the last delivery ago
def last_delivery(df):
    print("Apply function: last_delivery")
    group=df.groupby(['SHOP_ID', 'OBJECT_ID'])
    return df.assign(LAST_DEL=group.PERIOD_TS.shift(0)-group.PERIOD_TS.shift(1))


# Relative price current period
def rel_price_current_period(df, q=0.5):
    print("Apply function: rel_price_current_period")
    price=df.PRICE_ORG+df.PRICE_CALC*(df.PRICE_ORG==0)
    price_for_agg=price.replace([0, 1, 100], np.nan)
    price_cp=df.assign(PRICE=price_for_agg)\
               .groupby(['OBJECT_ID', 'PERIOD_TS'])\
               .agg({'PRICE': lambda x: np.nanquantile(x, q)})
    
    return df.join(price_cp, on=['OBJECT_ID', 'PERIOD_TS'], how='left')\
             .assign(REL_PRICE_CP=lambda x: (price/(x.PRICE+1e-8)))\
             .fillna(value={'REL_PRICE_CP': 1.0})\
             .drop(columns='PRICE')

# Relative price previous period
def rel_price_prev_periods(df, q=0.5):
    print("Apply function: rel_price_prev_periods")
    price=df.PRICE_ORG+df.PRICE_CALC*(df.PRICE_ORG==0)
    price_for_agg=price.replace([0, 1, 100], np.nan)
    price_pp=df.assign(PRICE=price_for_agg)\
               .groupby(['OBJECT_ID', 'SHOP_ID'])\
               .agg({'PRICE': lambda x: np.nanquantile(x, q)})

    return df.join(price_pp, on=['OBJECT_ID', 'SHOP_ID'], how='left')\
             .assign(REL_PRICE_PP=lambda x: (price/(x.PRICE+1e-8)))\
             .fillna(value={'REL_PRICE_PP': 1.0})\
             .drop(columns='PRICE')

# Relative sales previous period
def rel_sales_prev_periods(df, q=0.5):
    print("Apply function: rel_sales_prev_periods")
    sales_pp=df.assign(SALES=df.SALES_VOLUME)\
               .groupby(['OBJECT_ID', 'SHOP_ID'])\
               .agg({'SALES': lambda x: np.nanquantile(x, q)})

    return df.join(sales_pp, on=['OBJECT_ID', 'SHOP_ID'], how='left')\
             .assign(REL_SALES_PP=lambda x: (x.SALES_VOLUME/(x.SALES+1e-8)))\
             .fillna(value={'REL_SALES_PP': 1.0})\
             .drop(columns='SALES')

#converte date
def convert_date_in(df):
    print("Apply function: convert_date_in")
    return df.assign(DATE_IN=pd.to_datetime(df.DATE_IN, dayfirst=True))

###  Past periods rolling window

In [3]:
keys={'TURNOVER': 'TO',\
      'SALES_VOLUME': 'SV',\
      'PRICE_ORG': 'PR',\
      'OBJECT_ID': 'OB',\
      'SHOP_ID': 'SH',\
      'BRAND': 'BR'}

In [4]:
# Shift Turnover, Sales Volume and Price org um jeweils eine Woche zurück - lag(1) 
def past_period(df, periods=[1, 2, 3], group=None, q=0.5, target=['TURNOVER', 'SALES_VOLUME', 'PRICE_ORG']):
    print("Apply function: past_period")
    if group is None:
        df_select=df.set_index('PERIOD_TS')[target+['SHOP_ID', 'OBJECT_ID']]
        
        df_join=df
        for p in periods:
            col_names=[keys[t]+'_PP'+str(p) for t in target]
            df_join=df_join.join(df_select.shift(freq=pd.Timedelta(str(p*7)+' days'))\
                                     .set_index(['SHOP_ID', 'OBJECT_ID'], append=True)\
                                     .rename(columns={t: col_name for t, col_name in zip(target, col_names)}),\
                                 on=['PERIOD_TS', 'SHOP_ID', 'OBJECT_ID'], how='left')

        return df_join.fillna(0)
    else:
        agg_name='{:.0f}'.format(q*100)
        df_agg=df.replace({t: 0 for t in target}, np.nan)\
                 .groupby(group+['PERIOD_TS'], as_index=False)[target]\
                 .agg(lambda x: np.nanquantile(x, q=q))\
                 .set_index('PERIOD_TS')
        
        df_agg_shift=[]
        for p in periods:            
            col_names=['V_'+agg_name+'_'+keys[t]+'_PP'+str(p)+'_'+'_'.join([keys[g] for g in group]) for t in target]

            df_agg_shift.append(df_agg.shift(freq=pd.Timedelta(str(p*7)+' days'))\
                                       .set_index(group, append=True)\
                                       .rename(columns={t: col_name for t, col_name in zip(target, col_names)}))

        return df.join(pd.concat(df_agg_shift, axis=1), on=['PERIOD_TS']+group, how='left').fillna(0) 

### Present period aggregation

In [5]:
# lower and higher quantile
def leakage_free_agg(x, q):
    lower=x.quantile(q-10e-6, interpolation='lower')
    higher=x.quantile(q, interpolation='higher')
    sep=(x<higher)
    return sep*higher+~sep*lower

def current_period(df, group=['OBJECT_ID'], q=0.5, target=['TURNOVER', 'SALES_VOLUME', 'PRICE_ORG'], crop='2018-08-05 00:00:00'):
#Group aggregation of values in curr period. Record targets are excluded from aggregation to prevent leakage.
    print("Apply function: current_period")
    agg_name='{:.0f}'.format(q*100)
    col_names=['V_'+agg_name+'_'+keys[t]+'_CP_'+'_'.join([keys[g] for g in group]) for t in target]  
    nan_replace=['V_'+agg_name+'_'+keys[t]+'_PP1_'+'_'.join([keys[g] for g in group]) for t in target]
    
    quantile=df[df.DATE_IN<pd.to_datetime(crop)]\
               .replace({'PRICE_ORG': 0}, np.nan)\
               .groupby(group+['PERIOD_TS']).filter(lambda x: len(x) >1)\
               .groupby(group+['PERIOD_TS'])[target]\
               .apply(lambda x: leakage_free_agg(x, q))\
               .rename(columns={t: col_name for t, col_name in zip(target, col_names)})
    
    return df.join(quantile).fillna(value={col_name: df[pp] for pp, col_name in zip(nan_replace, col_names)})

### Other Features 

In [6]:
# NEW_OB_SH: Show if, previous record exists
def new(df, group=['OBJECT_ID', 'SHOP_ID']):
#True/False whether group has no previous records. Also True if no previous records exist.
    print("Apply function: new")
    col_name='NEW_'+'_'.join([keys[g] for g in group])
    new_item=df.groupby(group)\
               .agg({'PERIOD_TS': 'min'})\
               .rename(columns={'PERIOD_TS': 'EARLIEST_TS'})  
    return df.join(new_item, on=group, how='left')\
             .assign(**{col_name: lambda x: x['PERIOD_TS']<=x['EARLIEST_TS']})\
             .drop(columns='EARLIEST_TS')


# idicate previous fill type
def prev_fill_type(df):
    print("Apply function: prev_fill_type")
    ohe_fill_type=pd.get_dummies(df.sort_values(by='PERIOD_TS')\
                    .groupby(['OBJECT_ID', 'SHOP_ID'])['FILL_TYPE']\
                    .shift(1), prefix='FILL_TYPE', prefix_sep='_',)
    
    return df.join(ohe_fill_type)

### Lisa's Functions 

In [7]:
#delete columns with no variance
def del_constant_var (df):
    df=df.loc[:, (df != df.iloc[0]).any()]
    return df

In [8]:
# ^2 for all relevant number variables
def exponent_two(df, excluded_cols,exponent_three=True):
    
    df_help= df.drop(excluded_cols, axis=1)
    
    df_num=df_help.select_dtypes(include=['number'])
    df_quad=np.power(df_num,2)
    df_columns = list(df_quad.columns.values)
    for col in df_columns:
        df_quad.rename(columns={col:"{}_quadr".format(col)}, inplace=True)
   

    if exponent_three ==True:
        df_three=np.power(df_num,3)
        df_columns2 = list(df_three.columns.values)
        for col in df_columns2:
            df_three.rename(columns={col:"{}_three".format(col)}, inplace=True)

        df= pd.concat([df,df_quad,df_three], axis=1)
        return df
    else:
        df=pd.concat([df,df_quad], axis=1)
        return df

In [9]:
#dummiefy non numeric vars
def get_dummies (df):
    
    df_num=df.select_dtypes(exclude=['number'])
    #df_num=df_num.drop(excluded_cols, axis=1)
    df_num_columns = list(df_num.columns.values)
    i=0
    
    for var in df_num_columns:
        dummies = pd.get_dummies(df[var],drop_first=True, prefix=df_num.columns[i])
        del df[var]
        df=pd.concat([df,dummies],axis=1)
        i=i+1
    return df

### IOP Object Classes 

In [10]:
# Globals to read data
class IOP_DATA:
    def __init__(self, country):               
        self.path_src='./data/IOP_PTV_'+country
        self.path_item_md='./data/master_data/ItemDetailMaster_PTV_'+country+'.csv'
        self.path_shop_md='./data/master_data/IOP_'+country+'_DeliveryTypesShopsMaster.csv'
        self.path_period_md='./data/master_data/IOP_DeliveryPeriods.csv'
        self.df_width=0    
    
    def load_data(self):
        print('INFO: ---Loading data frame.---')
        if 'DE' in self.path_src:
            self.df = pd.concat([pd.read_csv(self.path_src+'1.csv', sep=';', decimal=','),\
                                 pd.read_csv(self.path_src+'2.csv', sep=';', decimal=',')])
        else:
            self.df = pd.read_csv(self.path_src+'.csv', sep=';', decimal=',')
        self.df_width=self.df.shape[1]
        print('INFO: Loaded data frame. Records: {}'.format(len(self.df)))
        

    # apply pre processing functions
    def pre_process(self):
        print('INFO: ---Pre-processing data frame.---')
        self.df=self.df.pipe(filter_old_seq)\
                       .pipe(filter_blind_items)\
                       .pipe(enrich_objects, self.path_item_md)\
                       .pipe(enrich_shops, self.path_shop_md)\
                       .pipe(enrich_periods)\
                       .pipe(filter_period_type)\
                       .pipe(columns_fill_type)\
                       .pipe(last_delivery)\
                       .pipe(rel_price_current_period)\
                       .pipe(rel_price_prev_periods)\
                       .pipe(rel_sales_prev_periods)\
                       .pipe(convert_date_in)
        
                       
        self.df_width=self.df.shape[1]
        assert len(self.df)==len(self.df.drop_duplicates(['SHOP_ID', 'OBJECT_ID', 'PERIOD_STRING']))
        print('INFO: Preprocessed data frame. Records: {}'.format(len(self.df)))


    # apply build features functions        
    def build_features(self):
        print('INFO: ---Building features.---')
        self.df=self.df.pipe(new)\
                       .pipe(prev_fill_type)\
                       .pipe(past_period)\
                       .pipe(past_period, group=['OBJECT_ID'], q=0.75)\
                       .pipe(past_period, group=['OBJECT_ID'], q=0.5)\
                       .pipe(past_period, group=['OBJECT_ID'], q=0.25)\
                       .pipe(past_period, group=['SHOP_ID'], q=0.75)\
                       .pipe(past_period, group=['SHOP_ID'], q=0.5)\
                       .pipe(past_period, group=['SHOP_ID'], q=0.25)\
                       .pipe(current_period)\
                       .pipe(current_period, q=0.25)\
                       .pipe(current_period, q=0.75)\
                       .pipe(current_period, group=['SHOP_ID'])\
                       .pipe(current_period, group=['SHOP_ID'], q=0.25)\
                       .pipe(current_period, group=['SHOP_ID'], q=0.75)

In [11]:
de_iop=IOP_DATA('DE')
de_iop.load_data()
de_iop.pre_process()
de_iop.build_features()

INFO: ---Loading data frame.---
INFO: Loaded data frame. Records: 1455305
INFO: ---Pre-processing data frame.---
Apply function: filter_old_seq
INFO: Filter for latest delivery sequence. Filtered out 128356 records.
Apply function: filter_blind_items
INFO: Filtered out 953376 blind items.
Apply function: enrich_objects
INFO: Join item master data.
Apply function: enrich_shops
INFO: Join shop master data.
Apply function: enrich_periods
INFO: Join period master data and generate time stamps.
Apply function: filter_period_type
INFO: Filtered out 18042 with different ptype than W.
Apply function: columns_fill_type
INFO: Fill type - Num(SV & TO & PR)=253057, Num(TO & PR)=1368, Num(SV & PR)=18130, Num(SV & TO)=82001, Num(Invalid)=975.
Apply function: last_delivery
Apply function: rel_price_current_period


C:\Users\lamaag\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:3250: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


Apply function: rel_price_prev_periods
Apply function: rel_sales_prev_periods
Apply function: convert_date_in
INFO: Preprocessed data frame. Records: 355531
INFO: ---Building features.---
Apply function: new
Apply function: prev_fill_type
Apply function: past_period
Apply function: past_period
Apply function: past_period
Apply function: past_period
Apply function: past_period
Apply function: past_period
Apply function: past_period
Apply function: current_period
Apply function: current_period
Apply function: current_period
Apply function: current_period
Apply function: current_period
Apply function: current_period


In [12]:
data_de=de_iop.df
data_de.head(10).to_csv('./test.csv', sep=',') #[(test.PRED_PR!=0) & (test.PRICE_ORG!=0)]
pd.set_option('display.max_columns', None)

In [13]:
data_de.head()

,DELIVERY_PERIOD_ID,DELIVERY_TYPE_ID,DELIVERY_SEQ,SHOP_ID,OBJECT_ID,SALES_VOLUME,TURNOVER,PURCHASES_VOLUME,STOCK_OLD,STOCK_NEW,RENTALS,DATE_IN,PRICE_ORG,PRICE_CALC,DATE_IN_CENTRAL,IOP_SEQ,ITEM_ID,BRAND,ITEM_DESC,DETAIL_DESC,PERIOD_TYPE_ID,DEL_COMP_NAME,COMPANY_ID,COMPANY_NAME,GSNR,COUNTRY_CHANNEL_ID,COUNTRY_CHANNEL_DESC,PERIOD_TS,PERIOD_STRING,FILL_TYPE,LAST_DEL,REL_PRICE_CP,REL_PRICE_PP,REL_SALES_PP,NEW_OB_SH,FILL_TYPE_0.0,FILL_TYPE_1.0,FILL_TYPE_2.0,FILL_TYPE_3.0,FILL_TYPE_4.0,TO_PP1,SV_PP1,PR_PP1,TO_PP2,SV_PP2,PR_PP2,TO_PP3,SV_PP3,PR_PP3,V_75_TO_PP1_OB,V_75_SV_PP1_OB,V_75_PR_PP1_OB,V_75_TO_PP2_OB,V_75_SV_PP2_OB,V_75_PR_PP2_OB,V_75_TO_PP3_OB,V_75_SV_PP3_OB,V_75_PR_PP3_OB,V_50_TO_PP1_OB,V_50_SV_PP1_OB,V_50_PR_PP1_OB,V_50_TO_PP2_OB,V_50_SV_PP2_OB,V_50_PR_PP2_OB,V_50_TO_PP3_OB,V_50_SV_PP3_OB,V_50_PR_PP3_OB,V_25_TO_PP1_OB,V_25_SV_PP1_OB,V_25_PR_PP1_OB,V_25_TO_PP2_OB,V_25_SV_PP2_OB,V_25_PR_PP2_OB,V_25_TO_PP3_OB,V_25_SV_PP3_OB,V_25_PR_PP3_OB,V_75_TO_PP1_SH,V_75_SV_PP1_SH,V_75_PR_PP1_SH,V_75_TO_PP2_SH,V_75_SV_PP2_SH,V_75_PR_PP2_SH,V_75_TO_PP3_SH,V_75_SV_PP3_SH,V_75_PR_PP3_SH,V_50_TO_PP1_SH,V_50_SV_PP1_SH,V_50_PR_PP1_SH,V_50_TO_PP2_SH,V_50_SV_PP2_SH,V_50_PR_PP2_SH,V_50_TO_PP3_SH,V_50_SV_PP3_SH,V_50_PR_PP3_SH,V_25_TO_PP1_SH,V_25_SV_PP1_SH,V_25_PR_PP1_SH,V_25_TO_PP2_SH,V_25_SV_PP2_SH,V_25_PR_PP2_SH,V_25_TO_PP3_SH,V_25_SV_PP3_SH,V_25_PR_PP3_SH,V_50_TO_CP_OB,V_50_SV_CP_OB,V_50_PR_CP_OB,V_25_TO_CP_OB,V_25_SV_CP_OB,V_25_PR_CP_OB,V_75_TO_CP_OB,V_75_SV_CP_OB,V_75_PR_CP_OB,V_50_TO_CP_SH,V_50_SV_CP_SH,V_50_PR_CP_SH,V_25_TO_CP_SH,V_25_SV_CP_SH,V_25_PR_CP_SH,V_75_TO_CP_SH,V_75_SV_CP_SH,V_75_PR_CP_SH
23,2435,629,1,84977,112423284,1,38400,0,0,1,0,2018-06-04 12:39:26,39900,38400.0,04.06.2018 13:10:11,163143958.0,112423284.0,SONY,KDL-32WD755,KDL-32WD755,W,MEDIA/SATURN,647.0,MEDIA-SATURN,4291.0,1882.0,DE - Technical Superstores,2018-06-03,1822,1,0 days,1.000000,1.066845,1.000000,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39900.0,1.0,39900.0,39900.0,1.0,39900.0,79800.0,2.0,39900.0,54900.0,1.0,48800.0,29900.0,1.0,24900.0,89900.0,1.0,64900.0
24,2435,629,1,84977,116314887,1,15899,0,0,7,0,2018-06-04 12:39:25,15899,15899.0,04.06.2018 13:10:11,163143958.0,116314887.0,OK.,OLE 24650H-TB,OLE 24650H-TB,W,MEDIA/SATURN,647.0,MEDIA-SATURN,4291.0,1882.0,DE - Technical Superstores,2018-06-03,1822,1,0 days,0.828073,0.946397,0.666667,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19900.0,1.0,19700.0,15899.0,1.0,15899.0,20999.0,1.0,20899.0,54900.0,1.0,48800.0,31900.0,1.0,29900.0,89900.0,1.0,64900.0
25,2435,629,1,84977,121591972,1,55900,0,0,2,0,2018-06-04 12:39:24,55900,55900.0,04.06.2018 13:10:11,163143958.0,121591972.0,PANASONIC,TX-40EXW604,TX-40EXW604,W,MEDIA/SATURN,647.0,MEDIA-SATURN,4291.0,1882.0,DE - Technical Superstores,2018-06-03,1822,1,0 days,1.000000,1.000000,1.000000,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55900.0,1.0,55900.0,54400.0,1.0,55900.0,80000.0,2.0,55900.0,52900.0,1.0,39900.0,29900.0,1.0,24900.0,89900.0,1.0,64900.0
26,2435,629,1,84977,121756820,1,99900,0,0,4,0,2018-06-04 12:39:28,99900,99900.0,04.06.2018 13:10:11,163143958.0,121756820.0,SONY,KD-55XE8505,KD-55XE8505,W,MEDIA/SATURN,647.0,MEDIA-SATURN,4291.0,1882.0,DE - Technical Superstores,2018-06-03,1822,1,0 days,1.000000,1.000000,1.000000,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [17]:
data_de[data_de["SHOP_ID"]==6115767][["BRAND","COMPANY_NAME","COUNTRY_CHANNEL_DESC"]]

,BRAND,COMPANY_NAME,COUNTRY_CHANNEL_DESC


In [14]:
data_de.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355531 entries, 23 to 373270
Columns: 121 entries, DELIVERY_PERIOD_ID to V_75_PR_CP_SH
dtypes: bool(1), datetime64[ns](2), float64(90), int32(1), int64(12), object(9), timedelta64[ns](1), uint8(5)
memory usage: 315.3+ MB


In [15]:
data_de = data_de.sort_values(by=["SHOP_ID","OBJECT_ID", "DELIVERY_PERIOD_ID"]) # Evtl auch noch Object_ID
data_de.head()

,DELIVERY_PERIOD_ID,DELIVERY_TYPE_ID,DELIVERY_SEQ,SHOP_ID,OBJECT_ID,SALES_VOLUME,TURNOVER,PURCHASES_VOLUME,STOCK_OLD,STOCK_NEW,RENTALS,DATE_IN,PRICE_ORG,PRICE_CALC,DATE_IN_CENTRAL,IOP_SEQ,ITEM_ID,BRAND,ITEM_DESC,DETAIL_DESC,PERIOD_TYPE_ID,DEL_COMP_NAME,COMPANY_ID,COMPANY_NAME,GSNR,COUNTRY_CHANNEL_ID,COUNTRY_CHANNEL_DESC,PERIOD_TS,PERIOD_STRING,FILL_TYPE,LAST_DEL,REL_PRICE_CP,REL_PRICE_PP,REL_SALES_PP,NEW_OB_SH,FILL_TYPE_0.0,FILL_TYPE_1.0,FILL_TYPE_2.0,FILL_TYPE_3.0,FILL_TYPE_4.0,TO_PP1,SV_PP1,PR_PP1,TO_PP2,SV_PP2,PR_PP2,TO_PP3,SV_PP3,PR_PP3,V_75_TO_PP1_OB,V_75_SV_PP1_OB,V_75_PR_PP1_OB,V_75_TO_PP2_OB,V_75_SV_PP2_OB,V_75_PR_PP2_OB,V_75_TO_PP3_OB,V_75_SV_PP3_OB,V_75_PR_PP3_OB,V_50_TO_PP1_OB,V_50_SV_PP1_OB,V_50_PR_PP1_OB,V_50_TO_PP2_OB,V_50_SV_PP2_OB,V_50_PR_PP2_OB,V_50_TO_PP3_OB,V_50_SV_PP3_OB,V_50_PR_PP3_OB,V_25_TO_PP1_OB,V_25_SV_PP1_OB,V_25_PR_PP1_OB,V_25_TO_PP2_OB,V_25_SV_PP2_OB,V_25_PR_PP2_OB,V_25_TO_PP3_OB,V_25_SV_PP3_OB,V_25_PR_PP3_OB,V_75_TO_PP1_SH,V_75_SV_PP1_SH,V_75_PR_PP1_SH,V_75_TO_PP2_SH,V_75_SV_PP2_SH,V_75_PR_PP2_SH,V_75_TO_PP3_SH,V_75_SV_PP3_SH,V_75_PR_PP3_SH,V_50_TO_PP1_SH,V_50_SV_PP1_SH,V_50_PR_PP1_SH,V_50_TO_PP2_SH,V_50_SV_PP2_SH,V_50_PR_PP2_SH,V_50_TO_PP3_SH,V_50_SV_PP3_SH,V_50_PR_PP3_SH,V_25_TO_PP1_SH,V_25_SV_PP1_SH,V_25_PR_PP1_SH,V_25_TO_PP2_SH,V_25_SV_PP2_SH,V_25_PR_PP2_SH,V_25_TO_PP3_SH,V_25_SV_PP3_SH,V_25_PR_PP3_SH,V_50_TO_CP_OB,V_50_SV_CP_OB,V_50_PR_CP_OB,V_25_TO_CP_OB,V_25_SV_CP_OB,V_25_PR_CP_OB,V_75_TO_CP_OB,V_75_SV_CP_OB,V_75_PR_CP_OB,V_50_TO_CP_SH,V_50_SV_CP_SH,V_50_PR_CP_SH,V_25_TO_CP_SH,V_25_SV_CP_SH,V_25_PR_CP_SH,V_75_TO_CP_SH,V_75_SV_CP_SH,V_75_PR_CP_SH
35869,2435,40218,2,83700,106051143,-1,-85998,0,0,0,0,2018-08-31 19:07:43,85998,85998.0,31.08.2018 19:11:41,166855080.0,105690330.0,HISENSE,LTDN58K700XWTSEU3D,0,W,OTTO BAUR VERSAND WOCHEN,468.0,OTTO,5907.0,3889.0,DE - Pure Players Generalists,2018-06-03,1822,1,0 days,1.036139,1.000000,1.0,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120790,2437,40218,2,83700,111063065,-1,-24999,0,0,0,0,2018-08-31 19:34:10,24999,24999.0,31.08.2018 19:41:16,166855324.0,111062330.0,BLAUPUNKT,B40B148T2CS,B40B148T2CS,W,OTTO BAUR VERSAND WOCHEN,468.0,OTTO,5907.0,3889.0,DE - Pure Players Generalists,2018-06-17,1824,1,0 days,1.003976,1.041668,-1.0,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74700.0,3.0,26396.0,99600.0,4.0,25028.00,0.0,0.0,0.00,49800.0,2.0,26396.0,74700.0,3.0,25028.0,0.0,0.0,0.0,24900.0,1.0,26396.0,49800.0,2.0,25028.00,0.0,0.0,0.00,170478.50,4.0,71604.50,152248.25,2.75,70399.75,0.0,0.00,0.00,106328.0,2.0,51804.0,92848.0,2.0,50949.0,0.0,0.0,0.0,55540.50,1.0,39999.00,54247.50,1.0,39264.75,0.0,0.0,0.0,49800.0,2.0,26396.0,24900.0,1.0,26396.0,74700.0,3.0,26396.0,106328.0,2.0,51804.0,55540.5,1.0,39999.0,170478.5,4.0,71604.5
236503,2440,40218,2,83700,111063065,1,23276,0,0,0,0,2018-08-31 20:14:18,23276,23276.0,31.08.2018 20:21:11,166855592.0,111062330.0,BLAUPUNKT,B40B148T2CS,B40B148T2CS,W,OTTO BAUR VERSAND WOCHEN,468.0,OTTO,5907.0,3889.0,DE - Pure Players Generalists,2018-07-08,1827,1,21 days,0.934779,0.969874,1.0,False,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,-24999.0,-1.0,24999.0,49800.0,2.0,23670.0,49800.0,2.0,24926.00,49800.0,2.0,24784.25,24900.0,1.0,23670.0,24900.0,1.0,24926.0,24900.0,1.0,24569.5,24900.0,1.0,23670.0,24900.0,1.0,24926.00,24900.0,1.0,24354.75,443212.00,6.0,78847.00,303972.00,4.25,82796.25,229798.0,5.00,69999.00,133192.0,3.0,50666.0,139253.0,2.0,65923.5,134997.0,2.0,49900.0,61352.50,1.0,39457.50,46749.00,1.0,44999.00,59366.0,1.0,38999.0,24900.0,1.0,23670.0,24900.0,1.0,23670.0,49800.0,2.0,23670.0,133192.0,3.0,50666.0,61352.5,1.0,39457.5,443212.0,6.0,78847.0
273893,2441,40218,2,83700,111063065,1,23999,0,0,0,0,2018-08-31 20:29:25,23999,23999.0,31.0

In [16]:
data_de.index=range(len(data_de)) # fill in nrow data
data_de.head()

,DELIVERY_PERIOD_ID,DELIVERY_TYPE_ID,DELIVERY_SEQ,SHOP_ID,OBJECT_ID,SALES_VOLUME,TURNOVER,PURCHASES_VOLUME,STOCK_OLD,STOCK_NEW,RENTALS,DATE_IN,PRICE_ORG,PRICE_CALC,DATE_IN_CENTRAL,IOP_SEQ,ITEM_ID,BRAND,ITEM_DESC,DETAIL_DESC,PERIOD_TYPE_ID,DEL_COMP_NAME,COMPANY_ID,COMPANY_NAME,GSNR,COUNTRY_CHANNEL_ID,COUNTRY_CHANNEL_DESC,PERIOD_TS,PERIOD_STRING,FILL_TYPE,LAST_DEL,REL_PRICE_CP,REL_PRICE_PP,REL_SALES_PP,NEW_OB_SH,FILL_TYPE_0.0,FILL_TYPE_1.0,FILL_TYPE_2.0,FILL_TYPE_3.0,FILL_TYPE_4.0,TO_PP1,SV_PP1,PR_PP1,TO_PP2,SV_PP2,PR_PP2,TO_PP3,SV_PP3,PR_PP3,V_75_TO_PP1_OB,V_75_SV_PP1_OB,V_75_PR_PP1_OB,V_75_TO_PP2_OB,V_75_SV_PP2_OB,V_75_PR_PP2_OB,V_75_TO_PP3_OB,V_75_SV_PP3_OB,V_75_PR_PP3_OB,V_50_TO_PP1_OB,V_50_SV_PP1_OB,V_50_PR_PP1_OB,V_50_TO_PP2_OB,V_50_SV_PP2_OB,V_50_PR_PP2_OB,V_50_TO_PP3_OB,V_50_SV_PP3_OB,V_50_PR_PP3_OB,V_25_TO_PP1_OB,V_25_SV_PP1_OB,V_25_PR_PP1_OB,V_25_TO_PP2_OB,V_25_SV_PP2_OB,V_25_PR_PP2_OB,V_25_TO_PP3_OB,V_25_SV_PP3_OB,V_25_PR_PP3_OB,V_75_TO_PP1_SH,V_75_SV_PP1_SH,V_75_PR_PP1_SH,V_75_TO_PP2_SH,V_75_SV_PP2_SH,V_75_PR_PP2_SH,V_75_TO_PP3_SH,V_75_SV_PP3_SH,V_75_PR_PP3_SH,V_50_TO_PP1_SH,V_50_SV_PP1_SH,V_50_PR_PP1_SH,V_50_TO_PP2_SH,V_50_SV_PP2_SH,V_50_PR_PP2_SH,V_50_TO_PP3_SH,V_50_SV_PP3_SH,V_50_PR_PP3_SH,V_25_TO_PP1_SH,V_25_SV_PP1_SH,V_25_PR_PP1_SH,V_25_TO_PP2_SH,V_25_SV_PP2_SH,V_25_PR_PP2_SH,V_25_TO_PP3_SH,V_25_SV_PP3_SH,V_25_PR_PP3_SH,V_50_TO_CP_OB,V_50_SV_CP_OB,V_50_PR_CP_OB,V_25_TO_CP_OB,V_25_SV_CP_OB,V_25_PR_CP_OB,V_75_TO_CP_OB,V_75_SV_CP_OB,V_75_PR_CP_OB,V_50_TO_CP_SH,V_50_SV_CP_SH,V_50_PR_CP_SH,V_25_TO_CP_SH,V_25_SV_CP_SH,V_25_PR_CP_SH,V_75_TO_CP_SH,V_75_SV_CP_SH,V_75_PR_CP_SH
0,2435,40218,2,83700,106051143,-1,-85998,0,0,0,0,2018-08-31 19:07:43,85998,85998.0,31.08.2018 19:11:41,166855080.0,105690330.0,HISENSE,LTDN58K700XWTSEU3D,0,W,OTTO BAUR VERSAND WOCHEN,468.0,OTTO,5907.0,3889.0,DE - Pure Players Generalists,2018-06-03,1822,1,0 days,1.036139,1.000000,1.0,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2437,40218,2,83700,111063065,-1,-24999,0,0,0,0,2018-08-31 19:34:10,24999,24999.0,31.08.2018 19:41:16,166855324.0,111062330.0,BLAUPUNKT,B40B148T2CS,B40B148T2CS,W,OTTO BAUR VERSAND WOCHEN,468.0,OTTO,5907.0,3889.0,DE - Pure Players Generalists,2018-06-17,1824,1,0 days,1.003976,1.041668,-1.0,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74700.0,3.0,26396.0,99600.0,4.0,25028.00,0.0,0.0,0.00,49800.0,2.0,26396.0,74700.0,3.0,25028.0,0.0,0.0,0.0,24900.0,1.0,26396.0,49800.0,2.0,25028.00,0.0,0.0,0.00,170478.50,4.0,71604.50,152248.25,2.75,70399.75,0.0,0.00,0.00,106328.0,2.0,51804.0,92848.0,2.0,50949.0,0.0,0.0,0.0,55540.50,1.0,39999.00,54247.50,1.0,39264.75,0.0,0.0,0.0,49800.0,2.0,26396.0,24900.0,1.0,26396.0,74700.0,3.0,26396.0,106328.0,2.0,51804.0,55540.5,1.0,39999.0,170478.5,4.0,71604.5
2,2440,40218,2,83700,111063065,1,23276,0,0,0,0,2018-08-31 20:14:18,23276,23276.0,31.08.2018 20:21:11,166855592.0,111062330.0,BLAUPUNKT,B40B148T2CS,B40B148T2CS,W,OTTO BAUR VERSAND WOCHEN,468.0,OTTO,5907.0,3889.0,DE - Pure Players Generalists,2018-07-08,1827,1,21 days,0.934779,0.969874,1.0,False,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,-24999.0,-1.0,24999.0,49800.0,2.0,23670.0,49800.0,2.0,24926.00,49800.0,2.0,24784.25,24900.0,1.0,23670.0,24900.0,1.0,24926.0,24900.0,1.0,24569.5,24900.0,1.0,23670.0,24900.0,1.0,24926.00,24900.0,1.0,24354.75,443212.00,6.0,78847.00,303972.00,4.25,82796.25,229798.0,5.00,69999.00,133192.0,3.0,50666.0,139253.0,2.0,65923.5,134997.0,2.0,49900.0,61352.50,1.0,39457.50,46749.00,1.0,44999.00,59366.0,1.0,38999.0,24900.0,1.0,23670.0,24900.0,1.0,23670.0,49800.0,2.0,23670.0,133192.0,3.0,50666.0,61352.5,1.0,39457.5,443212.0,6.0,78847.0
3,2441,40218,2,83700,111063065,1,23999,0,0,0,0,2018-08-31 20:29:25,23999,23999.0,31.08.2018 20:32:06,166

In [17]:
print(data_de["SHOP_ID"].nunique())
print(len(data_de))

2041
355531


In [18]:
#replace PRICE_ORG with PRICE_CALC if PRICE_ORG is not available
data_de["PRICE_ORG"]=np.where((data_de["PRICE_ORG"]==0) & (data_de["PRICE_CALC"]!=0),data_de["PRICE_CALC"],data_de["PRICE_ORG"])

In [19]:
# generate compositional variable for AVs
UVs = ["TURNOVER","SALES_VOLUME"]
for i in UVs:
    comp_SV=data_de.groupby("SHOP_ID")[i].agg(["mean","median","var"])
    comp_SV["var"][np.isnan(comp_SV["var"])]=0
    data_de=pd.merge(data_de,comp_SV,how="left",on="SHOP_ID")
    data_de=data_de.rename(columns = {'mean':("mean_{}".format(i)),'median':("median_{}".format(i)),'var':("var_{}".format(i))})

In [20]:
#crate day-stamp for last delivery
last_del = []
for i in range(0,len(data_de["LAST_DEL"])):
    a=int(data_de["LAST_DEL"][i].days)
    last_del.append(a)
data_de= pd.concat([data_de,pd.Series(last_del, name="last_del")],axis=1)

In [21]:
# create day-delta between delivery and reprot date
delta_days=data_de["DATE_IN"]-data_de["PERIOD_TS"]
delta = []
for i in range(0,len(delta_days)):
    a=int(delta_days[i].days)
    delta.append(a)
data_de= pd.concat([data_de,pd.Series(delta, name="time_delta")],axis=1)

In [22]:
#delete unusable variables
df = data_de.drop(["ITEM_DESC","DETAIL_DESC","LAST_DEL","PERIOD_TS","DATE_IN","DATE_IN_CENTRAL","IOP_SEQ","DEL_COMP_NAME","COMPANY_ID","PERIOD_STRING","BRAND", "COMPANY_NAME","COUNTRY_CHANNEL_DESC"],axis=1)
df.head()

,DELIVERY_PERIOD_ID,DELIVERY_TYPE_ID,DELIVERY_SEQ,SHOP_ID,OBJECT_ID,SALES_VOLUME,TURNOVER,PURCHASES_VOLUME,STOCK_OLD,STOCK_NEW,RENTALS,PRICE_ORG,PRICE_CALC,ITEM_ID,PERIOD_TYPE_ID,GSNR,COUNTRY_CHANNEL_ID,FILL_TYPE,REL_PRICE_CP,REL_PRICE_PP,REL_SALES_PP,NEW_OB_SH,FILL_TYPE_0.0,FILL_TYPE_1.0,FILL_TYPE_2.0,FILL_TYPE_3.0,FILL_TYPE_4.0,TO_PP1,SV_PP1,PR_PP1,TO_PP2,SV_PP2,PR_PP2,TO_PP3,SV_PP3,PR_PP3,V_75_TO_PP1_OB,V_75_SV_PP1_OB,V_75_PR_PP1_OB,V_75_TO_PP2_OB,V_75_SV_PP2_OB,V_75_PR_PP2_OB,V_75_TO_PP3_OB,V_75_SV_PP3_OB,V_75_PR_PP3_OB,V_50_TO_PP1_OB,V_50_SV_PP1_OB,V_50_PR_PP1_OB,V_50_TO_PP2_OB,V_50_SV_PP2_OB,V_50_PR_PP2_OB,V_50_TO_PP3_OB,V_50_SV_PP3_OB,V_50_PR_PP3_OB,V_25_TO_PP1_OB,V_25_SV_PP1_OB,V_25_PR_PP1_OB,V_25_TO_PP2_OB,V_25_SV_PP2_OB,V_25_PR_PP2_OB,V_25_TO_PP3_OB,V_25_SV_PP3_OB,V_25_PR_PP3_OB,V_75_TO_PP1_SH,V_75_SV_PP1_SH,V_75_PR_PP1_SH,V_75_TO_PP2_SH,V_75_SV_PP2_SH,V_75_PR_PP2_SH,V_75_TO_PP3_SH,V_75_SV_PP3_SH,V_75_PR_PP3_SH,V_50_TO_PP1_SH,V_50_SV_PP1_SH,V_50_PR_PP1_SH,V_50_TO_PP2_SH,V_50_SV_PP2_SH,V_50_PR_PP2_SH,V_50_TO_PP3_SH,V_50_SV_PP3_SH,V_50_PR_PP3_SH,V_25_TO_PP1_SH,V_25_SV_PP1_SH,V_25_PR_PP1_SH,V_25_TO_PP2_SH,V_25_SV_PP2_SH,V_25_PR_PP2_SH,V_25_TO_PP3_SH,V_25_SV_PP3_SH,V_25_PR_PP3_SH,V_50_TO_CP_OB,V_50_SV_CP_OB,V_50_PR_CP_OB,V_25_TO_CP_OB,V_25_SV_CP_OB,V_25_PR_CP_OB,V_75_TO_CP_OB,V_75_SV_CP_OB,V_75_PR_CP_OB,V_50_TO_CP_SH,V_50_SV_CP_SH,V_50_PR_CP_SH,V_25_TO_CP_SH,V_25_SV_CP_SH,V_25_PR_CP_SH,V_75_TO_CP_SH,V_75_SV_CP_SH,V_75_PR_CP_SH,mean_TURNOVER,median_TURNOVER,var_TURNOVER,mean_SALES_VOLUME,median_SALES_VOLUME,var_SALES_VOLUME,last_del,time_delta
0,2435,40218,2,83700,106051143,-1,-85998,0,0,0,0,85998.0,85998.0,105690330.0,W,5907.0,3889.0,1,1.036139,1.000000,1.0,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176284.255172,99906.0,7.603981e+10,4.031034,2.0,58.569968,0,89
1,2437,40218,2,83700,111063065,-1,-24999,0,0,0,0,24999.0,24999.0,111062330.0,W,5907.0,3889.0,1,1.003976,1.041668,-1.0,True,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74700.0,3.0,26396.0,99600.0,4.0,25028.00,0.0,0.0,0.00,49800.0,2.0,26396.0,74700.0,3.0,25028.0,0.0,0.0,0.0,24900.0,1.0,26396.0,49800.0,2.0,25028.00,0.0,0.0,0.00,170478.50,4.0,71604.50,152248.25,2.75,70399.75,0.0,0.00,0.00,106328.0,2.0,51804.0,92848.0,2.0,50949.0,0.0,0.0,0.0,55540.50,1.0,39999.00,54247.50,1.0,39264.75,0.0,0.0,0.0,49800.0,2.0,26396.0,24900.0,1.0,26396.0,74700.0,3.0,26396.0,106328.0,2.0,51804.0,55540.5,1.0,39999.0,170478.5,4.0,71604.5,176284.255172,99906.0,7.603981e+10,4.031034,2.0,58.569968,0,75
2,2440,40218,2,83700,111063065,1,23276,0,0,0,0,23276.0,23276.0,111062330.0,W,5907.0,3889.0,1,0.934779,0.969874,1.0,False,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,-24999.0,-1.0,24999.0,49800.0,2.0,23670.0,49800.0,2.0,24926.00,49800.0,2.0,24784.25,24900.0,1.0,23670.0,24900.0,1.0,24926.0,24900.0,1.0,24569.5,24900.0,1.0,23670.0,24900.0,1.0,24926.00,24900.0,1.0,24354.75,443212.00,6.0,78847.00,303972.00,4.25,82796.25,229798.0,5.00,69999.00,133192.0,3.0,50666.0,139253.0,2.0,65923.5,134997.0,2.0,49900.0,61352.50,1.0,39457.50,46749.00,1.0,44999.00,59366.0,1.0,38999.0,24900.0,1.0,23670.0,24900.0,1.0,23670.0,49800.0,2.0,23670.0,133192.0,3.0,50666.0,61352.5,1.0,39457.5,443212.0,6.0,78847.0,176284.255172,99906.0,7.603981e+10,4.031034,2.0,58.569968,21,54
3,2441,40218,2,83700,111063065,1,23999,0,0,0,0,23999.0,23999.0,111062330.0,W,5907.0,3889.0,1,0.963815,1.000000,1.0,False,0,1,0,0,0,23276.0,1.0,23276.0,0.0,0.0,0.0,0.0,0.0,0.0,49800.0,2.0,23261.5,49800.0,2.0,23670.00,49800.0,2.0,24926.00,24900.0,1.0,23247.0,24900.0,1.0,23670.0,24900.0,1.0,24926.0,24900.0,1.0,23232.5,24900.0,1.0,23670.00,24900.0,1.0,24926.00,265795.00,4.0,79999.00,443212.00,6.00,78847.00,303972.0,4.25,82796.25,86197.0,2.0,59999.0,133192.0,3.0,5

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355531 entries, 0 to 355530
Columns: 116 entries, DELIVERY_PERIOD_ID to time_delta
dtypes: bool(1), float64(95), int32(1), int64(13), object(1), uint8(5)
memory usage: 311.8+ MB


In [24]:
df=del_constant_var(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355531 entries, 0 to 355530
Columns: 114 entries, DELIVERY_PERIOD_ID to time_delta
dtypes: bool(1), float64(95), int32(1), int64(12), uint8(5)
memory usage: 306.3 MB


In [25]:
df=exponent_two(df, ["PRICE_ORG","DELIVERY_PERIOD_ID","DELIVERY_TYPE_ID","DELIVERY_SEQ","OBJECT_ID","ITEM_ID","SHOP_ID"],exponent_three=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355531 entries, 0 to 355530
Columns: 220 entries, DELIVERY_PERIOD_ID to time_delta_quadr
dtypes: bool(1), float64(201), int32(1), int64(12), uint8(5)
memory usage: 593.9 MB


In [26]:
df=get_dummies(df)
df.head()

,DELIVERY_PERIOD_ID,DELIVERY_TYPE_ID,DELIVERY_SEQ,SHOP_ID,OBJECT_ID,SALES_VOLUME,TURNOVER,PURCHASES_VOLUME,STOCK_OLD,STOCK_NEW,PRICE_ORG,PRICE_CALC,ITEM_ID,GSNR,COUNTRY_CHANNEL_ID,FILL_TYPE,REL_PRICE_CP,REL_PRICE_PP,REL_SALES_PP,FILL_TYPE_0.0,FILL_TYPE_1.0,FILL_TYPE_2.0,FILL_TYPE_3.0,FILL_TYPE_4.0,TO_PP1,SV_PP1,PR_PP1,TO_PP2,SV_PP2,PR_PP2,TO_PP3,SV_PP3,PR_PP3,V_75_TO_PP1_OB,V_75_SV_PP1_OB,V_75_PR_PP1_OB,V_75_TO_PP2_OB,V_75_SV_PP2_OB,V_75_PR_PP2_OB,V_75_TO_PP3_OB,V_75_SV_PP3_OB,V_75_PR_PP3_OB,V_50_TO_PP1_OB,V_50_SV_PP1_OB,V_50_PR_PP1_OB,V_50_TO_PP2_OB,V_50_SV_PP2_OB,V_50_PR_PP2_OB,V_50_TO_PP3_OB,V_50_SV_PP3_OB,V_50_PR_PP3_OB,V_25_TO_PP1_OB,V_25_SV_PP1_OB,V_25_PR_PP1_OB,V_25_TO_PP2_OB,V_25_SV_PP2_OB,V_25_PR_PP2_OB,V_25_TO_PP3_OB,V_25_SV_PP3_OB,V_25_PR_PP3_OB,V_75_TO_PP1_SH,V_75_SV_PP1_SH,V_75_PR_PP1_SH,V_75_TO_PP2_SH,V_75_SV_PP2_SH,V_75_PR_PP2_SH,V_75_TO_PP3_SH,V_75_SV_PP3_SH,V_75_PR_PP3_SH,V_50_TO_PP1_SH,V_50_SV_PP1_SH,V_50_PR_PP1_SH,V_50_TO_PP2_SH,V_50_SV_PP2_SH,V_50_PR_PP2_SH,V_50_TO_PP3_SH,V_50_SV_PP3_SH,V_50_PR_PP3_SH,V_25_TO_PP1_SH,V_25_SV_PP1_SH,V_25_PR_PP1_SH,V_25_TO_PP2_SH,V_25_SV_PP2_SH,V_25_PR_PP2_SH,V_25_TO_PP3_SH,V_25_SV_PP3_SH,V_25_PR_PP3_SH,V_50_TO_CP_OB,V_50_SV_CP_OB,V_50_PR_CP_OB,V_25_TO_CP_OB,V_25_SV_CP_OB,V_25_PR_CP_OB,V_75_TO_CP_OB,V_75_SV_CP_OB,V_75_PR_CP_OB,V_50_TO_CP_SH,V_50_SV_CP_SH,V_50_PR_CP_SH,V_25_TO_CP_SH,V_25_SV_CP_SH,V_25_PR_CP_SH,V_75_TO_CP_SH,V_75_SV_CP_SH,V_75_PR_CP_SH,mean_TURNOVER,median_TURNOVER,var_TURNOVER,mean_SALES_VOLUME,median_SALES_VOLUME,var_SALES_VOLUME,last_del,time_delta,SALES_VOLUME_quadr,TURNOVER_quadr,PURCHASES_VOLUME_quadr,STOCK_OLD_quadr,STOCK_NEW_quadr,PRICE_CALC_quadr,GSNR_quadr,COUNTRY_CHANNEL_ID_quadr,FILL_TYPE_quadr,REL_PRICE_CP_quadr,REL_PRICE_PP_quadr,REL_SALES_PP_quadr,FILL_TYPE_0.0_quadr,FILL_TYPE_1.0_quadr,FILL_TYPE_2.0_quadr,FILL_TYPE_3.0_quadr,FILL_TYPE_4.0_quadr,TO_PP1_quadr,SV_PP1_quadr,PR_PP1_quadr,TO_PP2_quadr,SV_PP2_quadr,PR_PP2_quadr,TO_PP3_quadr,SV_PP3_quadr,PR_PP3_quadr,V_75_TO_PP1_OB_quadr,V_75_SV_PP1_OB_quadr,V_75_PR_PP1_OB_quadr,V_75_TO_PP2_OB_quadr,V_75_SV_PP2_OB_quadr,V_75_PR_PP2_OB_quadr,V_75_TO_PP3_OB_quadr,V_75_SV_PP3_OB_quadr,V_75_PR_PP3_OB_quadr,V_50_TO_PP1_OB_quadr,V_50_SV_PP1_OB_quadr,V_50_PR_PP1_OB_quadr,V_50_TO_PP2_OB_quadr,V_50_SV_PP2_OB_quadr,V_50_PR_PP2_OB_quadr,V_50_TO_PP3_OB_quadr,V_50_SV_PP3_OB_quadr,V_50_PR_PP3_OB_quadr,V_25_TO_PP1_OB_quadr,V_25_SV_PP1_OB_quadr,V_25_PR_PP1_OB_quadr,V_25_TO_PP2_OB_quadr,V_25_SV_PP2_OB_quadr,V_25_PR_PP2_OB_quadr,V_25_TO_PP3_OB_quadr,V_25_SV_PP3_OB_quadr,V_25_PR_PP3_OB_quadr,V_75_TO_PP1_SH_quadr,V_75_SV_PP1_SH_quadr,V_75_PR_PP1_SH_quadr,V_75_TO_PP2_SH_quadr,V_75_SV_PP2_SH_quadr,V_75_PR_PP2_SH_quadr,V_75_TO_PP3_SH_quadr,V_75_SV_PP3_SH_quadr,V_75_PR_PP3_SH_quadr,V_50_TO_PP1_SH_quadr,V_50_SV_PP1_SH_quadr,V_50_PR_PP1_SH_quadr,V_50_TO_PP2_SH_quadr,V_50_SV_PP2_SH_quadr,V_50_PR_PP2_SH_quadr,V_50_TO_PP3_SH_quadr,V_50_SV_PP3_SH_quadr,V_50_PR_PP3_SH_quadr,V_25_TO_PP1_SH_quadr,V_25_SV_PP1_SH_quadr,V_25_PR_PP1_SH_quadr,V_25_TO_PP2_SH_quadr,V_25_SV_PP2_SH_quadr,V_25_PR_PP2_SH_quadr,V_25_TO_PP3_SH_quadr,V_25_SV_PP3_SH_quadr,V_25_PR_PP3_SH_quadr,V_50_TO_CP_OB_quadr,V_50_SV_CP_OB_quadr,V_50_PR_CP_OB_quadr,V_25_TO_CP_OB_quadr,V_25_SV_CP_OB_quadr,V_25_PR_CP_OB_quadr,V_75_TO_CP_OB_quadr,V_75_SV_CP_OB_quadr,V_75_PR_CP_OB_quadr,V_50_TO_CP_SH_quadr,V_50_SV_CP_SH_quadr,V_50_PR_CP_SH_quadr,V_25_TO_CP_SH_quadr,V_25_SV_CP_SH_quadr,V_25_PR_CP_SH_quadr,V_75_TO_CP_SH_quadr,V_75_SV_CP_SH_quadr,V_75_PR_CP_SH_quadr,mean_TURNOVER_quadr,median_TURNOVER_quadr,var_TURNOVER_quadr,mean_SALES_VOLUME_quadr,median_SALES_VOLUME_quadr,var_SALES_VOLUME_quadr,last_del_quadr,time_delta_quadr,NEW_OB_SH_True
0,2435,40218,2,83700,106051143,-1,-85998,0,0,0,85998.0,85998.0,105690330.0,5907.0,3889.0,1,1.036139,1.000000,1.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [27]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.replace('.', '_')
df.columns = df.columns.str.replace('&', '_')
df.columns = df.columns.str.replace(':', '_')
df.columns = df.columns.str.replace('(', '_')
df.columns = df.columns.str.replace(')', '_')
df.columns = df.columns.str.replace('+', '_')
df.columns = df.columns.str.replace('*', '_')
df.columns = df.columns.str.replace('<', '_')
df.columns = df.columns.str.replace('>', '_')
df.columns = df.columns.str.replace('Ä', 'ae')
df.columns = df.columns.str.replace('Ö', 'oe')
df.columns = df.columns.str.replace('Ü', 'ue')

In [28]:
# Interaction-Terms for fix effect 
df["PRICE_CALC_PR_PP1"]=df["PRICE_CALC"]*df["PR_PP1"]
df["PRICE_CALC_PR_PP2"]=df["PRICE_CALC"]*df["PR_PP2"]
df["PRICE_CALC_PR_PP2"]=df["PRICE_CALC"]*df["PR_PP3"]
df["PRICE_CALC_last_del"]=df["PRICE_CALC"]*df["last_del"]
df["SALES_VOLUME_last_del"]=df["SALES_VOLUME"]*df["last_del"]

In [29]:
df.head()

,DELIVERY_PERIOD_ID,DELIVERY_TYPE_ID,DELIVERY_SEQ,SHOP_ID,OBJECT_ID,SALES_VOLUME,TURNOVER,PURCHASES_VOLUME,STOCK_OLD,STOCK_NEW,PRICE_ORG,PRICE_CALC,ITEM_ID,GSNR,COUNTRY_CHANNEL_ID,FILL_TYPE,REL_PRICE_CP,REL_PRICE_PP,REL_SALES_PP,FILL_TYPE_0_0,FILL_TYPE_1_0,FILL_TYPE_2_0,FILL_TYPE_3_0,FILL_TYPE_4_0,TO_PP1,SV_PP1,PR_PP1,TO_PP2,SV_PP2,PR_PP2,TO_PP3,SV_PP3,PR_PP3,V_75_TO_PP1_OB,V_75_SV_PP1_OB,V_75_PR_PP1_OB,V_75_TO_PP2_OB,V_75_SV_PP2_OB,V_75_PR_PP2_OB,V_75_TO_PP3_OB,V_75_SV_PP3_OB,V_75_PR_PP3_OB,V_50_TO_PP1_OB,V_50_SV_PP1_OB,V_50_PR_PP1_OB,V_50_TO_PP2_OB,V_50_SV_PP2_OB,V_50_PR_PP2_OB,V_50_TO_PP3_OB,V_50_SV_PP3_OB,V_50_PR_PP3_OB,V_25_TO_PP1_OB,V_25_SV_PP1_OB,V_25_PR_PP1_OB,V_25_TO_PP2_OB,V_25_SV_PP2_OB,V_25_PR_PP2_OB,V_25_TO_PP3_OB,V_25_SV_PP3_OB,V_25_PR_PP3_OB,V_75_TO_PP1_SH,V_75_SV_PP1_SH,V_75_PR_PP1_SH,V_75_TO_PP2_SH,V_75_SV_PP2_SH,V_75_PR_PP2_SH,V_75_TO_PP3_SH,V_75_SV_PP3_SH,V_75_PR_PP3_SH,V_50_TO_PP1_SH,V_50_SV_PP1_SH,V_50_PR_PP1_SH,V_50_TO_PP2_SH,V_50_SV_PP2_SH,V_50_PR_PP2_SH,V_50_TO_PP3_SH,V_50_SV_PP3_SH,V_50_PR_PP3_SH,V_25_TO_PP1_SH,V_25_SV_PP1_SH,V_25_PR_PP1_SH,V_25_TO_PP2_SH,V_25_SV_PP2_SH,V_25_PR_PP2_SH,V_25_TO_PP3_SH,V_25_SV_PP3_SH,V_25_PR_PP3_SH,V_50_TO_CP_OB,V_50_SV_CP_OB,V_50_PR_CP_OB,V_25_TO_CP_OB,V_25_SV_CP_OB,V_25_PR_CP_OB,V_75_TO_CP_OB,V_75_SV_CP_OB,V_75_PR_CP_OB,V_50_TO_CP_SH,V_50_SV_CP_SH,V_50_PR_CP_SH,V_25_TO_CP_SH,V_25_SV_CP_SH,V_25_PR_CP_SH,V_75_TO_CP_SH,V_75_SV_CP_SH,V_75_PR_CP_SH,mean_TURNOVER,median_TURNOVER,var_TURNOVER,mean_SALES_VOLUME,median_SALES_VOLUME,var_SALES_VOLUME,last_del,time_delta,SALES_VOLUME_quadr,TURNOVER_quadr,PURCHASES_VOLUME_quadr,STOCK_OLD_quadr,STOCK_NEW_quadr,PRICE_CALC_quadr,GSNR_quadr,COUNTRY_CHANNEL_ID_quadr,FILL_TYPE_quadr,REL_PRICE_CP_quadr,REL_PRICE_PP_quadr,REL_SALES_PP_quadr,FILL_TYPE_0_0_quadr,FILL_TYPE_1_0_quadr,FILL_TYPE_2_0_quadr,FILL_TYPE_3_0_quadr,FILL_TYPE_4_0_quadr,TO_PP1_quadr,SV_PP1_quadr,PR_PP1_quadr,TO_PP2_quadr,SV_PP2_quadr,PR_PP2_quadr,TO_PP3_quadr,SV_PP3_quadr,PR_PP3_quadr,V_75_TO_PP1_OB_quadr,V_75_SV_PP1_OB_quadr,V_75_PR_PP1_OB_quadr,V_75_TO_PP2_OB_quadr,V_75_SV_PP2_OB_quadr,V_75_PR_PP2_OB_quadr,V_75_TO_PP3_OB_quadr,V_75_SV_PP3_OB_quadr,V_75_PR_PP3_OB_quadr,V_50_TO_PP1_OB_quadr,V_50_SV_PP1_OB_quadr,V_50_PR_PP1_OB_quadr,V_50_TO_PP2_OB_quadr,V_50_SV_PP2_OB_quadr,V_50_PR_PP2_OB_quadr,V_50_TO_PP3_OB_quadr,V_50_SV_PP3_OB_quadr,V_50_PR_PP3_OB_quadr,V_25_TO_PP1_OB_quadr,V_25_SV_PP1_OB_quadr,V_25_PR_PP1_OB_quadr,V_25_TO_PP2_OB_quadr,V_25_SV_PP2_OB_quadr,V_25_PR_PP2_OB_quadr,V_25_TO_PP3_OB_quadr,V_25_SV_PP3_OB_quadr,V_25_PR_PP3_OB_quadr,V_75_TO_PP1_SH_quadr,V_75_SV_PP1_SH_quadr,V_75_PR_PP1_SH_quadr,V_75_TO_PP2_SH_quadr,V_75_SV_PP2_SH_quadr,V_75_PR_PP2_SH_quadr,V_75_TO_PP3_SH_quadr,V_75_SV_PP3_SH_quadr,V_75_PR_PP3_SH_quadr,V_50_TO_PP1_SH_quadr,V_50_SV_PP1_SH_quadr,V_50_PR_PP1_SH_quadr,V_50_TO_PP2_SH_quadr,V_50_SV_PP2_SH_quadr,V_50_PR_PP2_SH_quadr,V_50_TO_PP3_SH_quadr,V_50_SV_PP3_SH_quadr,V_50_PR_PP3_SH_quadr,V_25_TO_PP1_SH_quadr,V_25_SV_PP1_SH_quadr,V_25_PR_PP1_SH_quadr,V_25_TO_PP2_SH_quadr,V_25_SV_PP2_SH_quadr,V_25_PR_PP2_SH_quadr,V_25_TO_PP3_SH_quadr,V_25_SV_PP3_SH_quadr,V_25_PR_PP3_SH_quadr,V_50_TO_CP_OB_quadr,V_50_SV_CP_OB_quadr,V_50_PR_CP_OB_quadr,V_25_TO_CP_OB_quadr,V_25_SV_CP_OB_quadr,V_25_PR_CP_OB_quadr,V_75_TO_CP_OB_quadr,V_75_SV_CP_OB_quadr,V_75_PR_CP_OB_quadr,V_50_TO_CP_SH_quadr,V_50_SV_CP_SH_quadr,V_50_PR_CP_SH_quadr,V_25_TO_CP_SH_quadr,V_25_SV_CP_SH_quadr,V_25_PR_CP_SH_quadr,V_75_TO_CP_SH_quadr,V_75_SV_CP_SH_quadr,V_75_PR_CP_SH_quadr,mean_TURNOVER_quadr,median_TURNOVER_quadr,var_TURNOVER_quadr,mean_SALES_VOLUME_quadr,median_SALES_VOLUME_quadr,var_SALES_VOLUME_quadr,last_del_quadr,time_delta_quadr,NEW_OB_SH_True,PRICE_CALC_PR_PP1,PRICE_CALC_PR_PP2,PRICE_CALC_last_del,SALES_VOLUME_last_del
0,2435,40218,2,83700,106051143,-1,-85998,0,0,0,85998.0,85998.0,105690330.0,5907.0,3889.0,1,1.036139,1.000000,1.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.

### Save big data

In [30]:
df.to_csv("price_prep_final.csv")

### Draw 10% Sample 

In [31]:
Shop_id=pd.DataFrame(df.SHOP_ID.unique())
Shop_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2041 entries, 0 to 2040
Data columns (total 1 columns):
0    2041 non-null int64
dtypes: int64(1)
memory usage: 16.0 KB


In [32]:
Shop_id=pd.DataFrame(Shop_id.sample(frac=0.1, replace=False, random_state=1000))

In [33]:
Shop_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204 entries, 346 to 1042
Data columns (total 1 columns):
0    204 non-null int64
dtypes: int64(1)
memory usage: 3.2 KB


In [34]:
Shop_id.columns=["SHOP_ID"]
Shop_id[Shop_id["SHOP_ID"]==401216]

,SHOP_ID


In [35]:
data_small=pd.merge(Shop_id,df,how="left",on="SHOP_ID")
data_small.head()

,SHOP_ID,DELIVERY_PERIOD_ID,DELIVERY_TYPE_ID,DELIVERY_SEQ,OBJECT_ID,SALES_VOLUME,TURNOVER,PURCHASES_VOLUME,STOCK_OLD,STOCK_NEW,PRICE_ORG,PRICE_CALC,ITEM_ID,GSNR,COUNTRY_CHANNEL_ID,FILL_TYPE,REL_PRICE_CP,REL_PRICE_PP,REL_SALES_PP,FILL_TYPE_0_0,FILL_TYPE_1_0,FILL_TYPE_2_0,FILL_TYPE_3_0,FILL_TYPE_4_0,TO_PP1,SV_PP1,PR_PP1,TO_PP2,SV_PP2,PR_PP2,TO_PP3,SV_PP3,PR_PP3,V_75_TO_PP1_OB,V_75_SV_PP1_OB,V_75_PR_PP1_OB,V_75_TO_PP2_OB,V_75_SV_PP2_OB,V_75_PR_PP2_OB,V_75_TO_PP3_OB,V_75_SV_PP3_OB,V_75_PR_PP3_OB,V_50_TO_PP1_OB,V_50_SV_PP1_OB,V_50_PR_PP1_OB,V_50_TO_PP2_OB,V_50_SV_PP2_OB,V_50_PR_PP2_OB,V_50_TO_PP3_OB,V_50_SV_PP3_OB,V_50_PR_PP3_OB,V_25_TO_PP1_OB,V_25_SV_PP1_OB,V_25_PR_PP1_OB,V_25_TO_PP2_OB,V_25_SV_PP2_OB,V_25_PR_PP2_OB,V_25_TO_PP3_OB,V_25_SV_PP3_OB,V_25_PR_PP3_OB,V_75_TO_PP1_SH,V_75_SV_PP1_SH,V_75_PR_PP1_SH,V_75_TO_PP2_SH,V_75_SV_PP2_SH,V_75_PR_PP2_SH,V_75_TO_PP3_SH,V_75_SV_PP3_SH,V_75_PR_PP3_SH,V_50_TO_PP1_SH,V_50_SV_PP1_SH,V_50_PR_PP1_SH,V_50_TO_PP2_SH,V_50_SV_PP2_SH,V_50_PR_PP2_SH,V_50_TO_PP3_SH,V_50_SV_PP3_SH,V_50_PR_PP3_SH,V_25_TO_PP1_SH,V_25_SV_PP1_SH,V_25_PR_PP1_SH,V_25_TO_PP2_SH,V_25_SV_PP2_SH,V_25_PR_PP2_SH,V_25_TO_PP3_SH,V_25_SV_PP3_SH,V_25_PR_PP3_SH,V_50_TO_CP_OB,V_50_SV_CP_OB,V_50_PR_CP_OB,V_25_TO_CP_OB,V_25_SV_CP_OB,V_25_PR_CP_OB,V_75_TO_CP_OB,V_75_SV_CP_OB,V_75_PR_CP_OB,V_50_TO_CP_SH,V_50_SV_CP_SH,V_50_PR_CP_SH,V_25_TO_CP_SH,V_25_SV_CP_SH,V_25_PR_CP_SH,V_75_TO_CP_SH,V_75_SV_CP_SH,V_75_PR_CP_SH,mean_TURNOVER,median_TURNOVER,var_TURNOVER,mean_SALES_VOLUME,median_SALES_VOLUME,var_SALES_VOLUME,last_del,time_delta,SALES_VOLUME_quadr,TURNOVER_quadr,PURCHASES_VOLUME_quadr,STOCK_OLD_quadr,STOCK_NEW_quadr,PRICE_CALC_quadr,GSNR_quadr,COUNTRY_CHANNEL_ID_quadr,FILL_TYPE_quadr,REL_PRICE_CP_quadr,REL_PRICE_PP_quadr,REL_SALES_PP_quadr,FILL_TYPE_0_0_quadr,FILL_TYPE_1_0_quadr,FILL_TYPE_2_0_quadr,FILL_TYPE_3_0_quadr,FILL_TYPE_4_0_quadr,TO_PP1_quadr,SV_PP1_quadr,PR_PP1_quadr,TO_PP2_quadr,SV_PP2_quadr,PR_PP2_quadr,TO_PP3_quadr,SV_PP3_quadr,PR_PP3_quadr,V_75_TO_PP1_OB_quadr,V_75_SV_PP1_OB_quadr,V_75_PR_PP1_OB_quadr,V_75_TO_PP2_OB_quadr,V_75_SV_PP2_OB_quadr,V_75_PR_PP2_OB_quadr,V_75_TO_PP3_OB_quadr,V_75_SV_PP3_OB_quadr,V_75_PR_PP3_OB_quadr,V_50_TO_PP1_OB_quadr,V_50_SV_PP1_OB_quadr,V_50_PR_PP1_OB_quadr,V_50_TO_PP2_OB_quadr,V_50_SV_PP2_OB_quadr,V_50_PR_PP2_OB_quadr,V_50_TO_PP3_OB_quadr,V_50_SV_PP3_OB_quadr,V_50_PR_PP3_OB_quadr,V_25_TO_PP1_OB_quadr,V_25_SV_PP1_OB_quadr,V_25_PR_PP1_OB_quadr,V_25_TO_PP2_OB_quadr,V_25_SV_PP2_OB_quadr,V_25_PR_PP2_OB_quadr,V_25_TO_PP3_OB_quadr,V_25_SV_PP3_OB_quadr,V_25_PR_PP3_OB_quadr,V_75_TO_PP1_SH_quadr,V_75_SV_PP1_SH_quadr,V_75_PR_PP1_SH_quadr,V_75_TO_PP2_SH_quadr,V_75_SV_PP2_SH_quadr,V_75_PR_PP2_SH_quadr,V_75_TO_PP3_SH_quadr,V_75_SV_PP3_SH_quadr,V_75_PR_PP3_SH_quadr,V_50_TO_PP1_SH_quadr,V_50_SV_PP1_SH_quadr,V_50_PR_PP1_SH_quadr,V_50_TO_PP2_SH_quadr,V_50_SV_PP2_SH_quadr,V_50_PR_PP2_SH_quadr,V_50_TO_PP3_SH_quadr,V_50_SV_PP3_SH_quadr,V_50_PR_PP3_SH_quadr,V_25_TO_PP1_SH_quadr,V_25_SV_PP1_SH_quadr,V_25_PR_PP1_SH_quadr,V_25_TO_PP2_SH_quadr,V_25_SV_PP2_SH_quadr,V_25_PR_PP2_SH_quadr,V_25_TO_PP3_SH_quadr,V_25_SV_PP3_SH_quadr,V_25_PR_PP3_SH_quadr,V_50_TO_CP_OB_quadr,V_50_SV_CP_OB_quadr,V_50_PR_CP_OB_quadr,V_25_TO_CP_OB_quadr,V_25_SV_CP_OB_quadr,V_25_PR_CP_OB_quadr,V_75_TO_CP_OB_quadr,V_75_SV_CP_OB_quadr,V_75_PR_CP_OB_quadr,V_50_TO_CP_SH_quadr,V_50_SV_CP_SH_quadr,V_50_PR_CP_SH_quadr,V_25_TO_CP_SH_quadr,V_25_SV_CP_SH_quadr,V_25_PR_CP_SH_quadr,V_75_TO_CP_SH_quadr,V_75_SV_CP_SH_quadr,V_75_PR_CP_SH_quadr,mean_TURNOVER_quadr,median_TURNOVER_quadr,var_TURNOVER_quadr,mean_SALES_VOLUME_quadr,median_SALES_VOLUME_quadr,var_SALES_VOLUME_quadr,last_del_quadr,time_delta_quadr,NEW_OB_SH_True,PRICE_CALC_PR_PP1,PRICE_CALC_PR_PP2,PRICE_CALC_last_del,SALES_VOLUME_last_del
0,379139,2444,629,1,99973088,-1,-195000,0,0,1,200000.0,195000.0,99973088.0,7853.0,1882.0,1,0.727405,1.0,1.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-279900.0,-1.0,279900.0,-254900.0,-1.0,334900.0,-239225.0,-1.25,194237.5,-279900.0,-1.0,279900.0,-299900.0,-1.0,319900.

In [36]:
data_small.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41450 entries, 0 to 41449
Columns: 224 entries, SHOP_ID to SALES_VOLUME_last_del
dtypes: float64(204), int32(1), int64(13), uint8(6)
memory usage: 69.3 MB


In [37]:
data_small["SHOP_ID"].nunique()

204

In [38]:
data_small.to_csv("price_data_small_final.csv")